In [ ]:
!pip install pymupdf4llm

In [ ]:
!pip install -U --quiet langchain-community

In [ ]:
!pip install --quiet langchain faiss-cpu sentence-transformers requests

In [ ]:
!pip install llama-index --no-cache-dir --quiet

# Import Modules

In [ ]:
!pip install --upgrade langchain


In [ ]:
import os
import pymupdf4llm
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_classic.chains.retrieval_qa.base import RetrievalQA

# API KEY

In [ ]:
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("openrouter-api-key")

# Load PDF and Convert to Markdown

In [ ]:
md_read = pymupdf4llm.LlamaMarkdownReader()
data = md_read.load_data("/content/drive/MyDrive/OPCNetBrokerUserGuide.pdf")
full_text = "\n".join(d.text for d in data if hasattr(d, 'text'))
print(f"Full text: {(full_text)}")

Full text: OPCNet Broker [®] User Guide

# **Integration Objects’**
## **Solution for Industrial Network** **Security and Connectivity** **OPCNet Broker [®] DA HDA AE**

#### Version 4.2 Rev.1

## **USER GUIDE**


customerservice@integrationobjects.com 1



OPCNet Broker [®] User Guide


OPCNet Broker [®] User Guide Version 4.2 Rev 1

Published September 2020

Copyright © 2003-2020 Integration Objects. All rights reserved.

No part of this document may be reproduced, stored in a retrieval system, translated, or transmitted, in
any form or by any means, electronic, mechanical, photocopying, recording, or otherwise, without the prior
written permission of Integration Objects.

Windows [®], Windows NT [® ] and .NET are registered trademarks of Microsoft Corporation.

OPCNet Broker is a registered trademark of Integration Objects.


OPC Foundation Self-Tested for Compliance logo is a trademark of the OPC Foundation and may be
used only by written permission of the OPC Foundation. Any unaut

# Split text into Chunks

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
docs = splitter.create_documents([full_text])

print(f"Number of Chunks: {len(docs)}")

lengths = [len(d.page_content) for d in docs]
print("Min length:" , min(lengths))
print("Max length:" , max(lengths))
print("Average length:" , sum(lengths) / len(lengths))

Number of Chunks: 170
Min length: 72
Max length: 796
Average length: 664.7352941176471


# Local Embeddings + FAISS Store

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs,embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

 # Set-up LLM API

In [ ]:
llm = ChatOpenAI(
    model="gpt-oss-120b",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.3
)

# Build RAG Chain

In [ ]:
from langchain_classic.chains.retrieval_qa.base import RetrievalQA
from langchain_classic.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are a helpful assistant that answers questions ONLY using the provided context"
        "Context: \n{context}\n\n"
        "Question: {question}\n\n"
        "If the context does not contain the necessary data to generate an answer, say 'The answer is not found in the provided document'"
        )
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)

# Ask Question

In [ ]:
print("RAG Chatbot Ready!")
print("Type 'exit' to quit.\n")

while True:
    q = input("You: ").strip()
    if q.lower() in ("exit", "quit"):
        break
    result = qa_chain(q)
    print("\nAssistant:", result["result"], "\n")


RAG Chatbot Ready!
Type 'exit' to quit.

You: hi

Assistant: The answer is not found in the provided document. 



KeyboardInterrupt: Interrupted by user